In [4]:
%load_ext sql

In [5]:

%sql sqlite:///myfile.db

In [6]:
%%sql

-- Select average revenue per employee by sector
SELECT sector, 
       avg(revenues/employees) AS avg_rev_employee
  FROM Fortune500
 GROUP BY sector
 -- Use the column alias to order the results
 ORDER BY avg_rev_employee limit 4;

 * sqlite:///myfile.db
Done.


sector,avg_rev_employee
"Hotels, Restaurants & Leisure",0.09498718151056815
Apparel,0.2786594297668007
Food & Drug Stores,0.307999504100602
Motor Vehicles & Parts,0.34252712424659526


In [7]:
%%sql
-- Divide unanswered_count by question_count
SELECT unanswered_count/question_count AS computed_pct, 
       -- What are you comparing the above quantity to?
       unanswered_pct
  FROM stackoverflow
 -- Select rows where question_count is not 0
 WHERE question_count != 0
 LIMIT 10;

 * sqlite:///myfile.db
Done.


computed_pct,unanswered_pct
0,0.001751857
0,0.000116972
0,5.8e-05
0,1.61e-05
0,0.000125312
0,0.012886449
0,0.007619406
0,1.23e-05
0,8.11e-05
0,0.000243743


In [18]:
%%sql
-- Select min, avg, max, and stddev of fortune500 profits
SELECT min(profits),
       max(profits),
       avg(profits)
       --,       stddev(profits)
  FROM fortune500;

 * sqlite:///myfile.db
Done.


min(profits),max(profits),"avg(profits) --, stddev(profits)"
-6177.0,45687.0,1783.4753507014022


In [17]:
%%sql
-- Select sector and summary measures of fortune500 profits
-- Add missing comma after avg(profits)
SELECT sector,
  min(profits),
  avg(profits),
  max(profits)
FROM fortune500
GROUP BY sector
ORDER BY avg(profits) limit 5;

 * sqlite:///myfile.db
Done.


sector,min(profits),avg(profits),max(profits)
Energy,-6177.0,10.444642857142838,7840.0
Materials,-440.0,272.4684210526316,1027.0
Engineering & Construction,15.0,390.1692307692308,911.8
Wholesalers,-199.4,391.2793103448276,2258.0
Retailing,-2221.0,991.7851063829781,13643.0


In [19]:
%%sql
-- Compute standard deviation of maximum values
SELECT --STDDEV(max_question_count),
       -- min
       MIN(max_question_count),
       -- max
       MAX(max_question_count),
       -- avg
       AVG(max_question_count)
  -- Subquery to compute max of question_count by tag
  FROM (SELECT tag,
               MAX(question_count) AS max_question_count
          FROM stackoverflow
         -- Compute max by tag
         GROUP BY tag) AS max_results; -- alias for subquery

 * sqlite:///myfile.db
Done.


MIN(max_question_count),MAX(max_question_count),AVG(max_question_count) -- Subquery to compute max of question_count by tag
30,1138658,52652.43396226415


In [20]:
%%sql
-- Truncate employees
SELECT trunc(employees, -5) AS employee_bin,
       -- Count number of companies with each truncated value
       count(*)
  FROM fortune500
 -- Use alias to group
 GROUP BY employee_bin
 -- Use alias to order
 ORDER BY employee_bin;

 * sqlite:///myfile.db
(sqlite3.OperationalError) wrong number of arguments to function trunc()
[SQL: -- Truncate employees
SELECT trunc(employees, -5) AS employee_bin,
       -- Count number of companies with each truncated value
       count(*)
  FROM fortune500
 -- Use alias to group
 GROUP BY employee_bin
 -- Use alias to order
 ORDER BY employee_bin;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [21]:
%%sql
-- Truncate employees
SELECT cast(employees / 100000 AS integer) * 100000 AS employee_bin,
       -- Count number of companies with each truncated value
       count(*)
  FROM fortune500
 -- Use alias to group
 GROUP BY employee_bin
 -- Use alias to order
 ORDER BY employee_bin;

 * sqlite:///myfile.db
Done.


employee_bin,count(*)
0,433
100000,35
200000,20
300000,7
400000,4
2300000,1


In [22]:
%%sql
-- Truncate employees
SELECT trunc(employees, -4) AS employee_bin,
       -- Count number of companies with each truncated value
       count(*)
  FROM fortune500
 -- Limit to which companies?
 WHERE employees < 100000
 -- Use alias to group
 GROUP BY employee_bin
 -- Use alias to order
 ORDER BY employee_bin;

 * sqlite:///myfile.db
(sqlite3.OperationalError) wrong number of arguments to function trunc()
[SQL: -- Truncate employees
SELECT trunc(employees, -4) AS employee_bin,
       -- Count number of companies with each truncated value
       count(*)
  FROM fortune500
 -- Limit to which companies?
 WHERE employees < 100000
 -- Use alias to group
 GROUP BY employee_bin
 -- Use alias to order
 ORDER BY employee_bin;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [23]:
%%sql
-- Truncate employees
SELECT CAST(employees/10000 AS INT)*10000 AS employee_bin,
       -- Count number of companies with each truncated value
       count(*)
  FROM fortune500
 -- Limit to which companies?
 WHERE employees < 100000
 -- Use alias to group
 GROUP BY employee_bin
 -- Use alias to order
 ORDER BY employee_bin;

 * sqlite:///myfile.db
Done.


employee_bin,count(*)
0,102
10000,108
20000,63
30000,42
40000,35
50000,31
60000,18
70000,18
80000,6
90000,10


In [25]:
%%sql
-- Select the min and max of question_count
SELECT MIN(question_count), 
       MAX(question_count)
  -- From what table?
  FROM stackoverflow
 -- For tag dropbox
 WHERE tag = 'dropbox';

 * sqlite:///myfile.db
Done.


MIN(question_count),MAX(question_count) -- From what table?
2315,3072


In [26]:
%%sql
-- Create lower and upper bounds of bins
SELECT generate_series(2200, 3050, 50) AS lower,
       generate_series(2250, 3100, 50) AS upper;

 * sqlite:///myfile.db
(sqlite3.OperationalError) no such function: generate_series
[SQL: -- Create lower and upper bounds of bins
SELECT generate_series(2200, 3050, 50) AS lower,
       generate_series(2250, 3100, 50) AS upper;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [28]:
%%sql
-- Bins created in Step 2
WITH bins AS (
      SELECT generate_series(2200, 3050, 50) AS lower,
             generate_series(2250, 3100, 50) AS upper),
     -- Subset stackoverflow to just tag dropbox (Step 1)
     dropbox AS (
      SELECT question_count 
        FROM stackoverflow
       WHERE tag='dropbox') 
-- Select columns for result
-- What column are you counting to summarize?
SELECT lower, upper, count(question_count) 
  FROM bins  -- Created above
       -- Join to dropbox (created above), 
       -- keeping all rows from the bins table in the join
       LEFT JOIN dropbox
       -- Compare question_count to lower and upper
         ON question_count >= lower 
        AND question_count < upper
 -- Group by lower and upper to count values in each bin
 GROUP BY lower, upper
 -- Order by lower to put bins in order
 ORDER BY lower;


 * sqlite:///myfile.db
(sqlite3.OperationalError) no such function: generate_series
[SQL: -- Bins created in Step 2
WITH bins AS (
      SELECT generate_series(2200, 3050, 50) AS lower,
             generate_series(2250, 3100, 50) AS upper),
     -- Subset stackoverflow to just tag dropbox (Step 1)
     dropbox AS (
      SELECT question_count 
        FROM stackoverflow
       WHERE tag='dropbox') 
-- Select columns for result
-- What column are you counting to summarize?
SELECT lower, upper, count(question_count) 
  FROM bins  -- Created above
       -- Join to dropbox (created above), 
       -- keeping all rows from the bins table in the join
       LEFT JOIN dropbox
       -- Compare question_count to lower and upper
         ON question_count >= lower 
        AND question_count < upper
 -- Group by lower and upper to count values in each bin
 GROUP BY lower, upper
 -- Order by lower to put bins in order
 ORDER BY lower;]
(Background on this error at: https://sqlalche.me/e/20/e3q

In [29]:
%%sql

-- Bins created in Step 2
WITH bins AS (
    SELECT 2200 AS lower, 2250 AS upper
    UNION ALL SELECT 2250, 2300
    UNION ALL SELECT 2300, 2350
    UNION ALL SELECT 2350, 2400
    UNION ALL SELECT 2400, 2450
    UNION ALL SELECT 2450, 2500
    UNION ALL SELECT 2500, 2550
    UNION ALL SELECT 2550, 2600
    UNION ALL SELECT 2600, 2650
    UNION ALL SELECT 2650, 2700
    UNION ALL SELECT 2700, 2750
    UNION ALL SELECT 2750, 2800
    UNION ALL SELECT 2800, 2850
    UNION ALL SELECT 2850, 2900
    UNION ALL SELECT 2900, 2950
    UNION ALL SELECT 2950, 3000
    UNION ALL SELECT 3000, 3050
    UNION ALL SELECT 3050, 3100
),
-- Subset stackoverflow to just tag dropbox (Step 1)
dropbox AS (
    SELECT question_count
    FROM stackoverflow
    WHERE tag='dropbox'
)
-- Select columns for result
-- What column are you counting to summarize?
SELECT lower, upper, count(question_count)
FROM bins -- Created above
-- Join to dropbox (created above),
-- keeping all rows from the bins table in the join
LEFT JOIN dropbox
-- Compare question_count to lower and upper
ON question_count >= lower
AND question_count < upper
-- Group by lower and upper to count values in each bin
GROUP BY lower, upper
-- Order by lower to put bins in order
ORDER BY lower;

 * sqlite:///myfile.db
Done.


lower,upper,count(question_count)
2200,2250,0
2250,2300,0
2300,2350,22
2350,2400,39
2400,2450,54
2450,2500,53
2500,2550,45
2550,2600,41
2600,2650,46
2650,2700,57


In [ ]:
# -- Correlation between revenues and profit
# SELECT CORR(revenues, profits) AS rev_profits,
#        -- Correlation between revenues and assets
#        CORR(revenues, assets) AS rev_assets,
#        -- Correlation between revenues and equity
#        CORR(revenues, equity) AS rev_equity 
#   FROM fortune500;

In [31]:
%%sql
-- Correlation between revenues and profit
SELECT 
    ROUND((SUM(revenues*profits) - SUM(revenues)*SUM(profits)/COUNT(*)) / (SUM(revenues*revenues) - SUM(revenues)*SUM(revenues)/COUNT(*)) , 2) AS rev_profits, 
    -- Correlation between revenues and assets
    ROUND((SUM(revenues*assets) - SUM(revenues)*SUM(assets)/COUNT(*)) / (SUM(revenues*revenues) - SUM(revenues)*SUM(revenues)/COUNT(*)) , 2) AS rev_assets, 
    -- Correlation between revenues and equity
    ROUND((SUM(revenues*equity) - SUM(revenues)*SUM(equity)/COUNT(*)) / (SUM(revenues*revenues) - SUM(revenues)*SUM(revenues)/COUNT(*)) , 2) AS rev_equity 
FROM fortune500

 * sqlite:///myfile.db
Done.


rev_profits,rev_assets,rev_equity
0.06,2.33,0.43


In [32]:
%%sql
-- What groups are you computing statistics by?
SELECT sector,
       -- Select the mean of assets with the avg function
       AVG(assets) AS mean,
       -- Select the median
       PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY assets) AS median
  FROM fortune500
 -- Computing statistics for each industry
 GROUP BY sector
 -- Order results by mean assets in descending order
 ORDER BY mean DESC;

 * sqlite:///myfile.db
(sqlite3.OperationalError) near "(": syntax error
[SQL: -- What groups are you computing statistics by?
SELECT sector,
       -- Select the mean of assets with the avg function
       AVG(assets) AS mean,
       -- Select the median
       PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY assets) AS median
  FROM fortune500
 -- Computing statistics for each industry
 GROUP BY sector
 -- Order results by mean assets in descending order
 ORDER BY mean DESC;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [34]:
%%sql
SELECT sector,
    AVG(assets) AS mean,
    (SELECT AVG(assets) 
     FROM fortune500 AS f2
     WHERE f2.sector = f1.sector AND (SELECT COUNT(*) 
                                      FROM fortune500 AS f3
                                      WHERE f3.sector = f2.sector AND f3.assets < f2.assets) 
                                     = (SELECT COUNT(*) 
                                         FROM fortune500 AS f3 
                                         WHERE f3.sector = f2.sector AND f3.assets <= f2.assets)) 
    AS median
FROM fortune500 AS f1
GROUP BY sector
ORDER BY mean DESC;

 * sqlite:///myfile.db
Done.


sector,mean,median
Financials,319245.09523809527,None
Telecommunications,114922.7,None
Motor Vehicles & Parts,59443.11111111111,None
Energy,48756.21052631579,None
Technology,48380.32558139535,None
Health Care,42078.89473684211,None
Industrials,40117.68421052631,None
Aerospace & Defense,31897.666666666668,None
Media,30876.272727272728,None
"Food, Beverages & Tobacco",29059.75,None


In [35]:
%%sql
-- To clear table if it already exists;
-- fill in name of temp table
DROP TABLE IF EXISTS profit80;

-- Create the temporary table
CREATE TEMP TABLE profit80 AS
  -- Select the two columns you need; alias as needed
  SELECT sector, 
         percentile_disc(0.8) WITHIN GROUP (ORDER BY profits) AS pct80
    -- What table are you getting the data from?
    FROM fortune500 
   -- What do you need to group by?
   GROUP BY sector;
   
-- See what you created: select all columns and rows 
-- from the table you created
SELECT * 
  FROM profit80;

 * sqlite:///myfile.db
Done.
(sqlite3.OperationalError) near "(": syntax error
[SQL: -- Create the temporary table
CREATE TEMP TABLE profit80 AS
  -- Select the two columns you need; alias as needed
  SELECT sector, 
         percentile_disc(0.8) WITHIN GROUP (ORDER BY profits) AS pct80
    -- What table are you getting the data from?
    FROM fortune500 
   -- What do you need to group by?
   GROUP BY sector;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [37]:
%%sql
-- Code from previous step
DROP TABLE IF EXISTS profit80;

CREATE TEMP TABLE profit80 AS
  SELECT sector, 
         percentile_disc(0.8) WITHIN GROUP (ORDER BY profits) AS pct80
    FROM fortune500 
   GROUP BY sector;

-- Select columns, aliasing as needed
SELECT title, fortune500.sector, 
       profits, profits/pct80 AS ratio
-- What tables do you need to join?  
  FROM fortune500 
       LEFT JOIN profit80
-- How are the tables joined?
       ON fortune500.sector=profit80.sector
-- What rows do you want to select?
 WHERE profits > pct80;

 * sqlite:///myfile.db
Done.
(sqlite3.OperationalError) near "(": syntax error
[SQL: CREATE TEMP TABLE profit80 AS
  SELECT sector, 
         percentile_disc(0.8) WITHIN GROUP (ORDER BY profits) AS pct80
    FROM fortune500 
   GROUP BY sector;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [39]:
%%sql
-- To clear table if it already exists
DROP TABLE IF EXISTS startdates;

-- Create temp table syntax
CREATE TEMP TABLE startdates AS
-- Compute the minimum date for each what?
SELECT tag,
       min(date) AS mindate
  FROM stackoverflow
 -- What do you need to compute the min date for each tag?
 GROUP BY tag;
 
 -- Look at the table you created
 SELECT * 
   FROM startdates limit 4;

 * sqlite:///myfile.db
Done.
Done.
Done.


tag,mindate
actionscript,2016-01-01
actionscript-3,2016-01-01
amazon,2016-01-01
amazon-cloudformation,2016-09-01


In [42]:
%%sql
-- To clear table if it already exists
DROP TABLE IF EXISTS startdates;

CREATE TEMP TABLE startdates AS
SELECT tag, min(date) AS mindate
  FROM stackoverflow
 GROUP BY tag;
 
-- Select tag (Remember the table name!) and mindate
SELECT startdates.tag, 
       mindate, 
       -- Select question count on the min and max days
	   so_min.question_count AS min_date_question_count,
       so_max.question_count AS max_date_question_count,
       -- Compute the change in question_count (max- min)
       so_max.question_count - so_min.question_count AS change
  FROM startdates
       -- Join startdates to stackoverflow with alias so_min
       INNER JOIN stackoverflow AS so_min
          -- What needs to match between tables?
          ON startdates.tag = so_min.tag
         AND startdates.mindate = so_min.date
       -- Join to stackoverflow again with alias so_max
       INNER JOIN stackoverflow AS so_max
          -- Again, what needs to match between tables?
          ON startdates.tag = so_max.tag
         AND so_max.date = '2018-09-25' limit 5;

 * sqlite:///myfile.db
Done.
Done.
Done.


tag,mindate,min_date_question_count,max_date_question_count,change
paypal,2016-01-01,13296,18050,4754
amazon-elb,2016-09-01,576,1452,876
amazon-mws,2016-09-01,367,706,339
amazon-swf,2016-09-01,167,232,65
amazon-sns,2016-09-01,690,1400,710


In [44]:
%%sql
DROP TABLE IF EXISTS correlations;

-- Create temp table 
CREATE TEMP TABLE correlations AS
-- Select each correlation
SELECT 'profits':: AS measure,
       -- Compute correlations
       corr(profits, profits) AS profits,
       corr(profits, profits_change) AS profits_change,
       corr(profits, revenues_change) AS revenues_change
  FROM fortune500;

 * sqlite:///myfile.db
Done.
(sqlite3.OperationalError) no such function: corr
[SQL: -- Create temp table 
CREATE TEMP TABLE correlations AS
-- Select each correlation
SELECT 'profits' AS measure,
       -- Compute correlations
       corr(profits, profits) AS profits,
       corr(profits, profits_change) AS profits_change,
       corr(profits, revenues_change) AS revenues_change
  FROM fortune500;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
%%sql
DROP TABLE IF EXISTS correlations;

CREATE TEMP TABLE correlations AS
SELECT 'profits'::varchar AS measure,
       corr(profits, profits) AS profits,
       corr(profits, profits_change) AS profits_change,
       corr(profits, revenues_change) AS revenues_change
  FROM fortune500;

INSERT INTO correlations
SELECT 'profits_change'::varchar AS measure,
       corr(profits_change, profits) AS profits,
       corr(profits_change, profits_change) AS profits_change,
       corr(profits_change, revenues_change) AS revenues_change
  FROM fortune500;

INSERT INTO correlations
SELECT 'revenues_change'::varchar AS measure,
       corr(revenues_change, profits) AS profits,
       corr(revenues_change, profits_change) AS profits_change,
       corr(revenues_change, revenues_change) AS revenues_change
  FROM fortune500;

-- Select each column, rounding the correlations
SELECT measure, 
       round(profits::numeric, 2) AS profits,
       round(profits_change::numeric, 2) AS profits_change,
       round(revenues_change::numeric, 2) AS revenues_change
  FROM correlations;

In [ ]:
%%sql


In [ ]:
%%sql
